## Day 19: Not Enough Minerals

[![nbviewer](https://raw.githubusercontent.com/jupyter/design/master/logos/Badges/nbviewer_badge.svg)](https://nbviewer.org/github/mazharenko/AoC-2022/tree/HEAD/notebooks/day19/puzzle.ipynb)

### Part 1

This is promises to be handy to define a type for resources. Here, among others, we have operators to determine whether there are enough resources (`/>=`) or not (`/<`)

In [26]:
type Res = {Ore: int; Clay:int; Obsidian: int;}
with static member Empty = {Ore = 0; Clay = 0; Obsidian = 0}
     static member (-)(res1, res2) =
        { Ore = res1.Ore - res2.Ore; Clay = res1.Clay - res2.Clay; Obsidian = res1.Obsidian - res2.Obsidian  }
     static member (+)(res1, res2) =
        { Ore = res1.Ore + res2.Ore; Clay = res1.Clay + res2.Clay; Obsidian = res1.Obsidian + res2.Obsidian  }
     static member (*)(res1, num) =
        { Ore = res1.Ore * num; Clay = res1.Clay * num; Obsidian = res1.Obsidian * num}
     static member (*)(num, res1) = res1 * num
     static member (/<)(res1, res2) = 
        res1.Ore < res2.Ore || res1.Clay < res2.Clay || res1.Obsidian < res2.Obsidian 
     static member (/>=)(res1, res2) = 
        res1.Ore >= res2.Ore && res1.Clay >= res2.Clay && res1.Obsidian >= res2.Obsidian 

let maxRes res1 res2 = 
   {
      Ore = max res1.Ore res2.Ore
      Clay = max res1.Clay res2.Clay
      Obsidian = max res1.Obsidian res2.Obsidian
   }

let clay = { Clay = 1; Ore = 0; Obsidian = 0 }
let ore = { Clay = 0; Ore = 1; Obsidian = 0 }
let obsidian = { Clay = 0; Ore = 0; Obsidian = 1 }

In [27]:
type Cost = Res
type Bp = {Id: int; OreRobot: Cost; ClayRobot: Cost; ObsidianRobot: Cost; GeodeRobot: Cost}
with member this.MaxRes = [ this.OreRobot; this.ClayRobot; this.ObsidianRobot; this.GeodeRobot ] |> List.reduce maxRes


In [28]:
#load "../common/common.fsx"

let parseBlueprint bp =
    match bp with 
    | Regex "Blueprint (\d+): Each ore robot costs (\d+) ore. Each clay robot costs (\d+) ore. Each obsidian robot costs (\d+) ore and (\d+) clay. Each geode robot costs (\d+) ore and (\d+) obsidian."
        [bp; oreOreCost; clayOreCost; obsidianOreCost; obsidianClayCost; geodeOreCost; geodeObsidianCost] -> 
            {Id = int bp; OreRobot = {Ore = int oreOreCost; Clay = 0; Obsidian = 0};
                ClayRobot = {Ore = int clayOreCost; Clay = 0; Obsidian = 0};
                ObsidianRobot = {Ore = int obsidianOreCost; Clay = int obsidianClayCost; Obsidian = 0};
                GeodeRobot = {Ore = int geodeOreCost; Clay = 0; Obsidian = int geodeObsidianCost}
             }

In [29]:
#!value --name sampleRaw --from-file ./data_sample.txt

In [30]:
#!share sampleRaw --from value
let sampleBps = Pattern1.read parseBlueprint sampleRaw
sampleBps

index Id OreRobot ClayRobot ObsidianRobot GeodeRobot MaxRes 0 1 Ore Clay Obsidian 4 0 0 Ore Clay Obsidian 2 0 0 Ore Clay Obsidian 3 14 0 Ore Clay Obsidian 2 0 7 Ore Clay Obsidian 4 14 7 1 2 Ore Clay Obsidian 2 0 0 Ore Clay Obsidian 3 0 0 Ore Clay Obsidian 3 8 0 Ore Clay Obsidian 3 0 12 Ore Clay Obsidian 3 8 12

This notebook relies on a breadth-first search algorithm implementation from the `bfs.fsx` script file. 

In this problem we have a graph of decicions taken during each minute. We traverse all possible decision paths reachable in 24 minutes.

In [31]:
type State = { 
    TimeLeft: int
    Bp: Bp
    Res: Res
    Geodes: int
    OreRobots: int
    ClayRobots: int
    ObsidianRobots: int
    GeodeRobots: int
} 
with 
    member this.PerMinute = clay * this.ClayRobots + ore * this.OreRobots + obsidian * this.ObsidianRobots
    member this.InTheEnd = this.Res + this.PerMinute * this.TimeLeft

To reduce the number of possible states some optimizations are introduced:
1. If there are enough resources to build a geode bot, this is the only correct choice.
2. There is no point in building a bot if there are already enough of them to build any kind of bots every minute.
3. There is no point in building a bot if we have already saved enough of the corresponding resource to build any kind of bots every minute.
4. There is no point in saving resources for a certain bot if there no bot produces resources for it.

In [32]:
#load "../common/bfs.fsx"
open Bfs
open Bfs.Custom

let adj : Adjacency<State> = 
    fun oldState -> 
        if (oldState.TimeLeft = 0)
        then []
        else 
            let state = {
                oldState
                    with Res = oldState.Res + oldState.PerMinute
                         Geodes = oldState.Geodes + oldState.GeodeRobots
                         TimeLeft = oldState.TimeLeft - 1
            }
            let maxRes = state.Bp.MaxRes

            if (oldState.Res />= state.Bp.GeodeRobot) // 1
            then 
                [ {state with Res = state.Res - state.Bp.GeodeRobot; GeodeRobots = state.GeodeRobots + 1} ]
            else 
                let alt = 
                    [
                        if oldState.Res />= state.Bp.ObsidianRobot
                        then 
                            if state.ObsidianRobots < maxRes.Obsidian && state.InTheEnd.Obsidian < maxRes.Obsidian * oldState.TimeLeft // 2 && 3
                            then 
                                yield {state with Res = state.Res - state.Bp.ObsidianRobot; ObsidianRobots = state.ObsidianRobots + 1}
                        elif state.InTheEnd />= state.Bp.ObsidianRobot //4
                        then yield state
                            
                        if oldState.Res />= state.Bp.OreRobot
                        then 
                            if state.OreRobots < maxRes.Ore && state.InTheEnd.Ore < maxRes.Ore * oldState.TimeLeft // 2 && 3
                            then 
                                yield {state with Res = state.Res - state.Bp.OreRobot; OreRobots = state.OreRobots + 1}
                        elif state.InTheEnd />= state.Bp.OreRobot //4
                        then yield state

                        if oldState.Res />= state.Bp.ClayRobot
                        then
                            if state.ClayRobots < maxRes.Clay && state.InTheEnd.Clay < maxRes.Clay * oldState.TimeLeft // 2 && 3
                            then 
                                yield {state with Res = state.Res - state.Bp.ClayRobot; ClayRobots = state.ClayRobots + 1}
                        elif state.InTheEnd />= state.Bp.ClayRobot // 4
                        then yield state
                        
                    ] |> List.distinct
                if alt |> List.isEmpty && state.InTheEnd />= state.Bp.GeodeRobot
                then [ state ]
                else alt


let target : Target<State> = 
    fun state -> false

let settings = { VisitedKey = fun s -> s.TimeLeft, s.Geodes, s.GeodeRobots, s.Res, s.ObsidianRobots, s.OreRobots, s.ClayRobots }


In [33]:
let initState bp time = 
    { TimeLeft = time; Bp = bp; Res = Res.Empty; Geodes = 0; OreRobots = 1; ClayRobots = 0; ObsidianRobots = 0; GeodeRobots = 0 }
let maxGeodes state = 
    let (NotFound(paths)) = findPath settings { Adjacency = adj } state target
    paths 
    |> Seq.map List.head 
    |> Seq.map (fun s -> s.Geodes)
    |> Seq.max


In [34]:
#!value --name actualRaw --from-file ./data_actual.txt

In [35]:
#!share actualRaw --from value
let actualBps = Pattern1.read parseBlueprint actualRaw


In [36]:
#!time
actualBps
|> Array.map (fun bp -> bp.Id * maxGeodes (initState bp 24))
|> Array.sum


2301

Wall time: 38002.7861ms

### Part 2

In [37]:
#!time
actualBps
|> Array.take 3
|> Array.map (fun bp -> maxGeodes (initState bp 32))
|> Array.reduce (*)

10336

Wall time: 86519.3432ms